# working in MLOps with multiple classification models and tracking results 

In [23]:
#import required libraries
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# The code below  generates a binary classification dataset with a 90:10 class imbalance using make_classification. The output np.unique(y, return_counts=True) confirms the distribution of samples across the two classes.

In [24]:
x, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

Split the dataset into training and testing sets

In [25]:
x_train,x_test,y_train,y_test  =train_test_split(x,y,test_size=0.3,random_state=42)

# Training with Logistic regression classifier

In [26]:
log_reg = LogisticRegression(C=1,solver='liblinear')
log_reg.fit(x_train,y_train)
y_pred_log_reg  =log_reg.predict(x_test)
print(classification_report(y_test,y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       270
           1       0.78      0.60      0.68        30

    accuracy                           0.94       300
   macro avg       0.87      0.79      0.82       300
weighted avg       0.94      0.94      0.94       300



# Training with Random forest classifier

In [27]:
rf_clf = RandomForestClassifier(n_estimators=30,max_depth=3)
rf_clf.fit(x_train,y_train)
y_pred_rf = rf_clf.predict(x_test)
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       1.00      0.63      0.78        30

    accuracy                           0.96       300
   macro avg       0.98      0.82      0.88       300
weighted avg       0.96      0.96      0.96       300



# Training with XGBoost

In [28]:
xgb_clf = XGBClassifier(use_label_encoder=False,eval_metric= 'logloss')
xgb_clf.fit(x_train,y_train)
y_pred_xgb = xgb_clf.predict(x_test)
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.88      0.77      0.82        30

    accuracy                           0.97       300
   macro avg       0.93      0.88      0.90       300
weighted avg       0.97      0.97      0.97       300



# USING SMOTE
# below code applies SMOTETomek, which combines over-sampling of the minority class using SMOTE and under-sampling of the majority class using Tomek Links. It helps to balance the class distribution in the training data. The np.unique function shows the updated sample counts for each class after resampling.
# Handel class imbalance using SMOTETomek then again train dataset with XGBoost

In [29]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=0)
x_train_res,y_train_res = smt.fit_resample(x_train,y_train)

np.unique(y_train_res,return_counts=True)

#Import SMOTETomek: Use from imblearn.combine import SMOTETomek to bring in the technique that combines SMOTE and Tomek Links for balancing class distribution.
#Initialize SMOTETomek: Create an instance with a fixed random state for reproducibility, e.g., smote_tomek = SMOTETomek(random_state=42).
#Apply fit_resample: Use X_train_res, y_train_res = smote_tomek.fit_resample(X_train, y_train) to generate a new balanced dataset.
#Check Class Distribution: Use np.unique(y_train_res, return_counts=True) to verify the new class distribution after resampling.
#Confirm Balance: The output should show equal counts for both classes, indicating successful balancing.

(array([0, 1]), array([614, 614]))

In [30]:
xgb_clf = XGBClassifier(use_label_encoder=False,eval_metric='logloss')
xgb_clf.fit(x_train_res,y_train_res)
y_pred_xgb = xgb_clf.predict(x_test)
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.83      0.83      0.83        30

    accuracy                           0.97       300
   macro avg       0.91      0.91      0.91       300
weighted avg       0.97      0.97      0.97       300



#  Tracking  all 4 above Experiments using MLFlow

# Models that are used below  Explained:

Logistic Regression

A linear model for binary classification.

C=1: Regularization parameter (higher = less regularization).

solver='liblinear': Good for small datasets and binary classification.

Random Forest

An ensemble of decision trees using bagging.

n_estimators=30: Builds 30 trees.

max_depth=3: Limits tree depth to reduce overfitting and improve generalization.

XGBoost Classifier

A powerful gradient boosting algorithm.

use_label_encoder=False: Disables deprecated label encoder (avoids warnings).

eval_metric='logloss': Uses log loss to evaluate performance during training.

XGBoost Classifier With SMOTE

Same model as above but trained on resampled (balanced) data using SMOTETomek.

(x_train_res, y_train_res): Balanced training data after applying SMOTE + Tomek Links.

Helps improve prediction performance on imbalanced datasets.

In [31]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (x_train_res, y_train_res),
        (x_test, y_test)
    )
]

In [32]:
reports = []

for model_name, model, train_set, test_set in models:
    x_train = train_set[0]
    y_train = train_set[1]
    x_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

#  Above Loop Breakdown (Line-by-Line):

for model_name, model, train_set, test_set in models:

Iterates through each model tuple defined earlier.

Unpacks the model name, the model object, and train/test datasets.

X_train = train_set[0] and y_train = train_set[1]

Extracts the training features and labels.

X_test = test_set[0] and y_test = test_set[1]

Extracts the test features and labels.

model.fit(X_train, y_train)

Trains the model using the training data.

y_pred = model.predict(X_test)

Uses the trained model to predict labels on the test set.

report = classification_report(y_test, y_pred, output_dict=True)

Generates a detailed performance report as a dictionary.

Includes metrics like precision, recall, F1-score, and support.

reports.append(report)

Appends the classification report to the reports list for later use or comparison.

In [33]:
import mlflow #Brings in MLflow, a tool to track and manage machine learning projects.
import mlflow.sklearn #Adds support to save and track scikit-learn models with MLflow.
import mlflow.xgboost #Adds support to save and track XGBoost models with MLflow.

In [34]:
# Initializing MLFLOW
mlflow.set_experiment("imbalanced classification")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/08/22 12:56:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/22 12:56:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/816397754637035625/runs/1e0e892a9eff410e89a3fca6dab2bfde
🧪 View experiment at: http://localhost:5000/#/experiments/816397754637035625


2025/08/22 12:56:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/22 12:56:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/816397754637035625/runs/3d5a9685979a4ae1bd73f7e6d6a0cf5e
🧪 View experiment at: http://localhost:5000/#/experiments/816397754637035625


2025/08/22 12:56:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/22 12:56:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/816397754637035625/runs/2efb1466c0904ba28d39322d85c9ba72
🧪 View experiment at: http://localhost:5000/#/experiments/816397754637035625


2025/08/22 12:56:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/22 12:56:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/816397754637035625/runs/3a6497351abe41c8b4e9c88023cd2ac3
🧪 View experiment at: http://localhost:5000/#/experiments/816397754637035625
